In [ ]:
with open("names.txt", "r") as file:
    words = file.read().splitlines()

In [ ]:
words[:10]

In [ ]:
len(words)

In [ ]:
b = {}
for w in words:
    chs = ["<S>"] + list(w) + ["<E>"]

    for ch1,ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1


In [ ]:
sorted(b.items(), key=lambda item: -item[1])

In [ ]:
%pip install matplotlib
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
%pip install torch torchvision torchaudio

In [ ]:
import torch

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)

In [ ]:
chars = sorted(list(set(''.join(words))))
stoi = {s : i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}

In [ ]:
N = torch.zeros((27, 27), dtype=torch.int32)
for w in words:
    chs = ["."] + list(w) + ["."]

    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
plt.figure(figsize=(16, 16))
plt.imshow(N, cmap="Blues")
for i in range(27):
    for j in range(27):
        chstr = itos[i] + itos[j]
        plt.text(j,i,chstr, ha="center", va="bottom", color="black", fontsize=8)
        plt.text(j,i,N[i,j].item(), ha="center", va="top", color="black", fontsize=8)
        
plt.axis("off")

In [ ]:
N[0]


In [ ]:
p = N[0].float()
p = p / p.sum()
p

In [ ]:
import datetime
g = torch.Generator().manual_seed(int(datetime.datetime.now().timestamp())) #fotr more random
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itos[ix]


In [ ]:
P = (N+1).float()
P = P / P.sum(dim=1, keepdim=True)

In [ ]:
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    out = []
    ix = 0
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
        
    print(''.join(out))
    

In [ ]:
log_likelihood = 0.0
n = 0
for w in words:
    chs = ["."] + list(w) + ["."]

    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1

nll = -log_likelihood / n
nll.item()

In [ ]:
xs, ys = [], []


for w in words[:1]:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)


In [ ]:
xs

In [ ]:
ys

In [ ]:
import torch.nn.functional as F


In [ ]:
xenc = F.one_hot(xs, num_classes=27).float()
yenc = F.one_hot(ys, num_classes=27).float()

In [ ]:
xenc.dtype

In [ ]:
g = torch.Generator().manual_seed(2147483647)  # for more random
W = torch.randn((27, 27), generator=g, requires_grad=True)


In [ ]:
logits = (xenc @ W) # Log Counts
counts = logits.exp()
probs = counts / counts.sum(dim=1, keepdim=True)
loss = -probs[torch.arange(5), ys].log().mean()
loss

In [ ]:
W.grad = None
loss.backward()
W.data += -100.0 * W.grad

In [ ]:
nlls = torch.zeros(5)
for i in range(5):
    x = xs[i].item()
    y = ys[i].item()
    print("-"* 20)
    print(f"bigram example {i+1}: {itos[x]}-{itos[y]} (indexes {x}-{y})")
    print("input to the neural network:", x)
    print("output probabilities:", probs[i])
    print("actual next character:", y)
    p = probs[i, y]
    print("probability of the actual next character:", p.item())
    logp = torch.log(p)
    print("log likelihood of the actual next character:", logp.item())
    nll = -logp
    print("negative log likelihood:", nll.item())
    nlls[i] = nll.item()
    
print("=" * 20)
print("Average negative log likelihood:", nlls.mean().item())

In [ ]:
probs.shape
    


In [ ]:
# Final

In [ ]:
xs, ys = [], []

for w in words:
    chs = ["."] + list(w) + ["."]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print(num)

g = torch.Generator().manual_seed(2147483647)  # for more random
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [ ]:
for k in range(20):
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = (xenc @ W)  # Log Counts
    counts = logits.exp()
    probs = counts / counts.sum(dim=1, keepdim=True)
    loss = -probs[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    W.grad = None
    loss.backward()

    W.data += -100.0 * W.grad


In [ ]:
g = torch.Generator().manual_seed(2147483647)
for i in range(5):
    out = []
    ix = 0
    while True:
        xenc = F.one_hot(torch.tensor(ix), num_classes=27).float()
        logits = (xenc @ W)  # Log Counts
        counts = logits.exp()
        p = counts / counts.sum(0, keepdim=True)
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break

    print(''.join(out))